In [2]:
import yaml
import sympy
import numpy as np
import math
# Semidefinite Programming solver
import cvxopt
# https://pypi.org/project/PyLMI-SDP is used to formulate LMIs
from lmi_sdp import LMI_PD as LMI, lmi_to_coeffs, to_cvxopt
from tf import transformations as tf
from robot_model import adjoint

np.set_printoptions(suppress=True, precision=1)  # print tiny values as zero

In [3]:
from IPython.display import Math
import lmi_sdp
lmi_sdp.init_lmi_latex_printing()

def show(lmis, vars=None):
	if vars is not None:
		lmis = [lmi.expanded(vars) for lmi in lmis]
	latex = r"\begin{{{env}}}{content}\end{{{env}}}".\
		format(env="gather", content=r" \\".join([sympy.latex(lmi) for lmi in lmis]))
	return Math(latex)

In [4]:
# Compute rotation matrix to rotate one vector onto another one
def rotation_from_vectors(src, tgt):
	src = np.asarray(src, dtype=float)
	tgt = np.asarray(tgt, dtype=float)
	angle = math.acos(src.dot(tgt) / (np.linalg.norm(src) * np.linalg.norm(tgt)))
	axis = np.cross(src, tgt)
	axis *= 1. / np.linalg.norm(axis)  # normalize axis!
	return tf.rotation_matrix(angle, axis)

# Compute T^o_{c_i} from contact position and normal (in object frame)
def transform_object_to_contact(position, normal):
	# TODO: Replace this dummy implementation with a working one!
	return np.eye(4)

In [5]:
def visualize_grasp(file):
	import rospy
	from markers import arrow, box, cone, createPose
	from std_msgs.msg import Header, ColorRGBA
	from geometry_msgs.msg import Vector3, Point
	from visualization_msgs.msg import MarkerArray, Marker

	kwargs = dict(ns="object", color=ColorRGBA(1, 1, 1, 0.1),
	              header=Header(frame_id="world"))
	msg=MarkerArray()
	msg.markers.append(box(size=Vector3(1,2,2), pose=createPose(np.eye(4)), **kwargs))
	kwargs.update(ns="contact", color=ColorRGBA(1, 0, 0, 0.5))
	size = 0.5
	points=[]

	def process(idx, position=None, normal=None, type=None, friction=0, torsion=0):
		points.append(np.asarray(position))
		kwargs.update(id=idx)
		if type == "frictionless":
			T = rotation_from_vectors([1, 0, 0], normal)  # rviz arrow points along x-axis
			T[:3, 3] = np.asarray(position)
			msg.markers.append(arrow(len=size, pose=createPose(T), **kwargs))
		else:
			half_angle = math.atan(friction)
			T = transform_object_to_contact(position, normal)
			msg.markers.append(cone(half_angle, scale=size, pose=createPose(T), **kwargs))
	
	with open(file, "r") as stream:
		for i, contact in enumerate(yaml.safe_load(stream)):
			process(i, **contact)

	if len(points) == 2:
		# draw a line conecting both contact points
		kwargs.update(ns="line", id=100, color=ColorRGBA(0, 1, 0, 0.5))
		msg.markers.append(Marker(type=Marker.LINE_STRIP, scale=Vector3(0.01, 0, 0),
		                          points=[Point(*p) for p in points], pose=createPose(np.eye(4)), **kwargs))

	rospy.init_node("lmi")
	pub = rospy.Publisher("contact_markers", MarkerArray, queue_size=1, latch=True)
	pub.publish(msg)
	rospy.sleep(1)


visualize_grasp('contacts.yaml')

In [6]:
# Load contacts from file and generate the grasp matrix G and corresponding friction cones
def load_grasp(file):
	def contact(idx, position=None, normal=None, type=None, friction=0, torsion=0):
		if type == "frictionless":
			basis = np.array([[0, 0, 1, 0, 0, 0]]).T
			vars = sympy.symbols(["f^{}_z".format(idx)])
			LMIs = [LMI(vars[0])]  # fz > 0
		else:
			# Coulomb friction
			basis = np.vstack((np.identity(3), np.zeros((3, 3))))
			vars = sympy.symbols(["f^{0}_{1}".format(idx, comp) for comp in "xyz"])
			LMIs = [LMI(vars[0]*sympy.Matrix([[1, 0], [0, -1]]) + 
			            vars[1]*sympy.Matrix([[0, 1], [1, 0]]) +
			            vars[2]*sympy.Matrix([[friction, 0], [0, friction]]))]
			if type == "softfinger":
				basis = np.hstack((basis, np.array([[0, 0, 0, 0, 0, 1]]).T))
				vars += sympy.symbols([r"\tau^{}_z".format(idx)])
				LMIs += [LMI(vars[2]*sympy.Matrix([[friction, 0], [0, friction]]) +
				             vars[3]*sympy.Matrix([[0, 1], [1, 0]]))]

		T = transform_object_to_contact(position, normal)
		return adjoint(T, inverse=True).T.dot(basis), LMIs, vars

	with open(file, "r") as stream:
		contacts = yaml.safe_load(stream)
	
	G = np.zeros((6,0))  # start with empty grasp matrix
	FC = []  # list of friction cone constraints
	vars = []  # list of free variables
	for i, c in enumerate(contacts):
		Gi, FCi, vi = contact(i+1, **c)
		# collect data from all contacts
		G = np.hstack((G, Gi))
		FC += FCi
		vars += vi
	
	return G, FC, vars

In [7]:
# Rewrite LMIs from vars x into vars z using x = Az + b.
# Using the notation from theorem 2.18 of the lecture script
# https://lernraumplus.uni-bielefeld.de/pluginfile.php/592816/mod_resource/content/2/script.pdf#page=44
def transform_LMIs(LMIs, x, A, b=None):
	A = sympy.Array(np.atleast_2d(A))
	m, l = A.shape
	if b is None:
		b = np.zeros(m)
	else:
		b = np.atleast_1d(b)
	z = sympy.Array(sympy.symbols(["z_{}".format(i+1) for i in range(l)]))
	j = sympy.Idx("j", l)  # index running from 0 to l-1

	transformed = [LMI(sympy.Matrix(S0 + sum([b[i]*Si[i] for i in range(m)])) +
	                   sympy.Sum(z[j] * sympy.Matrix(sum([A[i,j]*Si[i] for i in range(m)])), j).doit())
	               for Si, S0 in lmi_to_coeffs(LMIs, x, split_blocks=True)]
	return transformed, z

In [8]:
def eval_force_closure(G, FC, vars):
	# TODO: Implement this!
	return False

In [9]:
# Load grasp
G, FC, vars = load_grasp('contacts.yaml')
display(G) # and show its grasp matrix
show(FC) # as well as friction constraints

array([[1., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.]])

<IPython.core.display.Math object>

In [10]:
# Example: Transform LMIs into new basis x = 2*z + 0
FC, vars = transform_LMIs(FC, vars, 2*np.eye(G.shape[1]), np.zeros(G.shape[1]))
show(FC)

<IPython.core.display.Math object>

In [11]:
eval_force_closure(G, FC, vars)

False